In [1]:
import pandas as pd # $ pip install pandas
import folium # conda install -c conda-forge folium
import plotly.graph_objects as go
import plotly.express as px
import os
# import chart_studio.plotly as py # conda install -c plotly plotly, conda install -c plotly chart-studio

Möglicher Aufbau der Arbeit:
- 1 Seite Einleitung und Problemstellung/Was wir rausfinden wollen
- 2 Seiten Beschreibung der Daten/der Datenquelle
- 3 Seiten Daten einlesen in MongoDB
- 6 Seiten Analyse und Auswertung der Daten
- 1 Seite Schluss und beantworten der Frage

# Standorte

In [2]:
m = folium.Map(location=[51.2099, 10.4571], zoom_start=6)
folium.Marker([53.07736868080147, 8.804847969315563], tooltip="Obernstraße, Bremen").add_to(m)
folium.Marker([51.047494158917196, 13.737369656599013], tooltip="dresden-prager straße").add_to(m)
folium.Marker([551.222185050102674, 6.779255270096054], tooltip="düsseldorf-königsallee ostseiten").add_to(m)
folium.Marker([50.11494292444826, 8.687410770070198], tooltip="frankfurt a.m.-zeil").add_to(m)
folium.Marker([51.480738258522834, 11.97536492962145], tooltip="halle (saale)-leipziger straße").add_to(m)
folium.Marker([53.55376220607088, 9.991887654803696], tooltip="hamburg-jungfernstieg").add_to(m)
folium.Marker([52.374956125557894, 9.737387597107245], tooltip="hannover-georgstraße").add_to(m)
folium.Marker([50.001027164285354, 8.270445585412894], tooltip="mainz-stadthausstraße").add_to(m)
folium.Marker([48.137685457750635, 11.573367770032743], tooltip="münchen-kaufingerstraße").add_to(m)
folium.Marker([54.088503508553394, 12.134802012486357], tooltip="rostock-kröpeliner straße").add_to(m)
folium.Marker([49.2359910863105, 6.993367970054943], tooltip="saarbrücken-bahnhofstraße").add_to(m)
folium.Marker([48.77820910005942, 9.177711579841167], tooltip="stuttgart-königstraßee").add_to(m)

m

# Daten einlesen

Um Daten einzulesen, müssen wir eine Verbindung mit der MongoDB erstellen.

In [9]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["HyStreet"]
hystreet = db["HyStreetData"]

In [15]:
addresses = hystreet.distinct("address")
addresses

['Bahnhofstraße (Mitte), Saarbrücken',
 'Georgstraße, Hannover',
 'Jungfernstieg, Hamburg',
 'Kaufingerstraße, München',
 'Kröpeliner Straße (West), Rostock',
 'Königsallee Ostseite (Süd), Düsseldorf',
 'Königstraße (Mitte), Stuttgart',
 'Obernstraße, Bremen',
 'Prager Straße (Nord), Dresden',
 'Stadthausstraße, Mainz',
 'Zeil (Mitte), Frankfurt a.M.']

# Daten analysieren

In [25]:
def sumOfYear(data, year):
    addresses = hystreet.distinct("address")
    result = []
    
    for address in addresses:
        queryResult = hystreet.find({ "$and" : [ {"address" : address }, {"year" : year }]})
        data = pd.DataFrame(list(queryResult))
        result.append((address, data["pedestrians"].sum()))
        
    return result

sumOfYear(data, 2021)

[('Bahnhofstraße (Mitte), Saarbrücken', 9839731),
 ('Georgstraße, Hannover', 14034597),
 ('Jungfernstieg, Hamburg', 7379766),
 ('Kaufingerstraße, München', 15517650),
 ('Kröpeliner Straße (West), Rostock', 6840266),
 ('Königsallee Ostseite (Süd), Düsseldorf', 3579864),
 ('Königstraße (Mitte), Stuttgart', 7581398),
 ('Obernstraße, Bremen', 4923375),
 ('Prager Straße (Nord), Dresden', 8298939),
 ('Stadthausstraße, Mainz', 3215351),
 ('Zeil (Mitte), Frankfurt a.M.', 12529295)]

In [ ]:
def sumOfYearPerMonth(data, year):
    result = []
    month = [
        ["January",1],
        ["February",2],
        ["March",3],
        ["April",4],
        ["May",5],
        ["June",6],
        ["July",7],
        ["August",8],
        ["September",9],
        ["October",10],
        ["November",11],
        ["December",12]
    ]

    for city in data:
        filteredByYear = city.loc[city['year'] == year]
        for i in month:
            filteredByMonth = filteredByYear.loc[city['month'] == i[1]]
            result.append((city['location'][0], str(i[0]) + " " + str(year), filteredByMonth['pedestrians'].sum()))

    return result

sumOfYearPerMonth(data, 2022)

In [ ]:
def sumOfYearPerWeekday(data, year):
    result = [
        
    ]
    weekdays = [
        'Monday',
        'Tuesday',
        'Wednesday',
        'Thursday',
        'Friday',
        'Saturday',
        'Sunday'
    ]
    for city in data:
        filteredByYear = city.loc[city['year'] == year]
        for day in weekdays:
            filteredByWeekday = filteredByYear.loc[city['weekday'] == day]
            result.append((city['location'][0], day, filteredByWeekday['pedestrians'].sum()))

    return result

sumOfYearPerWeekday(data, 2021)

In [ ]:
def sumOfYearPerDay(data, year):
    result = []

    for city in data:
        filteredByYear = city.loc[city['year'] == year]
        for i in range(1,13):
            filteredByMonth = filteredByYear.loc[city['month'] == i]
            #print(filteredByMonth)
            for d in range(1,32):
                #print(d)
                pedestriansPerDay = filteredByMonth.loc[filteredByMonth['day'] == d]
                if not pedestriansPerDay.empty:
                    #print(pedestriansPerDay)
                    result.append((city['location'][0], i, d, pedestriansPerDay['pedestrians'].sum()))

    return result

sumOfYearPerDay(data, 2020)

In [ ]:
sumOverAllYear = sumOfYear(data, 2022) + sumOfYear(data, 2021) + sumOfYear(data, 2020)
long_df = pd.DataFrame(sumOverAllYear, columns=['location', 'countp'])

sumPersonOverAllYears = 0
anzahlNuller = 0

for persons in range(0,len(long_df.index)):
    if long_df['countp'][persons] != 0:
        sumPersonOverAllYears = sumPersonOverAllYears + long_df['countp'][persons]
    else:
        anzahlNuller = anzahlNuller + 1

# durch anzahl der Zonen / Durch anzahl der Jahre / Durch anzahl der Jahre 
sumPersonForAZoneAYear = sumPersonOverAllYears / len(files) / 3
sumPersonforAMonth = sumPersonForAZoneAYear / 12
sumPersonforADay = sumPersonforAMonth / 30.44
print(sumPersonforADay)


# Bereinigung der 0 jahre (3 nuller mit durschnittperosnen durch das jahr)
# sumPersonForADayCleand = sumPersonforADay + (anzahlNuller * sumPersonforADay / (len(files) * 3)) 
# print(sumPersonForADayCleand)


# Daten visualisieren

In [ ]:
absPed2020 = sumOfYear(data, 2020)
fig = px.bar(absPed2020, x=[x[0] for x in absPed2020], y=[y[1] for y in absPed2020], title='Absolute Zahlen 2020', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
fig.show()

In [ ]:
absPed2021 = sumOfYear(data, 2021)
fig = px.bar(absPed2021, x=[x[0] for x in absPed2021], y=[y[1] for y in absPed2021], title='Absolute Zahlen 2021', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
fig.show()

In [ ]:
absPed2022 = sumOfYear(data, 2022)
fig2 = px.bar(absPed2022, x=[x[0] for x in absPed2022], y=[y[1] for y in absPed2022], title='Absolute Zahlen 2022', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
fig2.show()

In [ ]:
weeklyPed2022 = sumOfYearPerWeekday(data, 2022)

long_df = pd.DataFrame(weeklyPed2022, columns=['location', 'weekday', 'countp'])
print(long_df)
fig = px.bar(long_df, x='weekday', y='countp', color="location")
fig.show()

## Vergleich der Relation zw. Einwohneranzahl und Passanten pro Jahr (in Welcher Stadt wird viel gelaufen)

## Jahreszeiten Vergleich

Im Folgenden werden die Jahreszeiten zunächst anhand derer absoluten Zahlen verglichen. Dafür wird das Jahr in die folgenden Abschnitte unterteilt: *Winter* (Dezember, Januar, Februar), *Frühling* (März, April, Mai), *Sommer* (Juni, Juli, August) und *Herbst* (September, Oktober, November). Hier werden die **meteorologischen Jahrezeiten** verwendet.

In [ ]:
def getRowsOfPeriod(sYear, sMonth, sDay, eYear, eMonth, eDay, data):
    '''Returns a dataframe with all the rows between the given dates (The given dates are inclusive).'''
    df = data.copy()
    for i, city in enumerate(df):
        city = city.loc[(city.year >= sYear) & (city.year <= eYear)]
        
        city = city.drop( city[(city.year == sYear) & (city.month < sMonth) ].index)
        city = city.drop( city[(city.year == eYear) & (city.month > eMonth) ].index)
    
        city = city.drop( city[(city.year == sYear) & (city.month == sMonth) & (city.day < sDay)].index)
        city = city.drop( city[(city.year == eYear) & (city.month == eMonth) & (city.day > eDay)].index)
    
        df[i] = city
    return df


def sumOfDf(dataframe):
    '''Returns the sum of the pedestrian column for a given dataframe.'''
    if not dataframe.empty:
        return dataframe['pedestrians'].sum()
    return 0


def sumOfDfs(dfList):
    '''Returns the sum of all the pedestrian sums for the given dataframes in the list'''
    result = 0
    for df in dfList:
        result += sumOfDf(df)
    return result
    

def seasonsOfYear(year, data):
    winterP1 = sumOfDfs(getRowsOfPeriod(year, 1, 1, year, 2, 30, data))
    winter = sumOfDfs(getRowsOfPeriod(year, 12, 1, year, 12, 31, data)) + winterP1
    spring = sumOfDfs(getRowsOfPeriod(year, 3, 1, year, 5, 31, data))
    sommer = sumOfDfs(getRowsOfPeriod(year, 6, 1, year, 8, 31, data))
    autumn = sumOfDfs(getRowsOfPeriod(year, 9, 1, year, 11, 31, data))
    
    result = [
        ('winter', winter),
        ('spring', spring),
        ('sommer', sommer),
        ('autumn', autumn)
    ]
    
    return pd.DataFrame(result, columns=['season', 'count'])



absSeasons2021 = seasonsOfYear(2021, data)
figSeasons2021 = px.bar(absSeasons2021, x='season', y='count', title='Vergleich Jahreszeiten', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
figSeasons2021.show()

Die Jahreszeiten können zusätzlich noch nach den einzelnen Monatem aufgeschlüsselt werden um einen genaueren Einblick zu schaffen.

In [ ]:
def monthsOfYear(year, data):
    '''Returns a dataframe containing all months in combination with the sum of all locations for that specific month'''
    df = data.copy()
    for idx, city in enumerate(df):
        df[idx] = city.loc[city.year == year]
    
    months = [0 for idx in range(12)]
    sMonth = ["Januar", "Februar", "März", "April", "Mai", "Juni", "Juli", "August", "September", "Oktober", "November", "Dezember"]
    
    for city in df:
        for month in range(1, 13):
            mDf = city.loc[city.month == month]
            months[month - 1] += sumOfDf(mDf)
            
    result = { 'Month' : sMonth, 'Count' : months }
            
    return pd.DataFrame(result)
    
    

absMonths2021 = monthsOfYear(2021, data)
figMonths2021 = px.bar(absMonths2021, x='Month', y='Count', title='Vergleich Monate 2021', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
figMonths2021.show()

In den Sommermonaten wegen dem Wetter so viel los und im November/Dezember wegen Weihnachten und Rabatten?

## Große Events vgl. zu normalen Tagen (einem durschnittstag) 

## Corona vergleich

Nun werden die Monate der Jahre 2020 bis Herbst 2022 verglichen. (deutlich zu erkennen: zwei Lockdowns)

In [ ]:
monthlyPed = sumOfYearPerMonth(data, 2020) + sumOfYearPerMonth(data, 2021) + sumOfYearPerMonth(data, 2022)
long_df = pd.DataFrame(monthlyPed, columns=['location', 'month', 'countp'])

fig = px.bar(long_df, x='month', y='countp', color="location")
fig.show()

Wöchentliche Fußgänger von Februar 2020 bis Mai 2020 (deutlich zu erkennen: Lockdown)

Tägliche Fußgänger im März 2020 pro Stadt (deutlich zu erkennen: Anfang des Lockdowns)

In [ ]:
sumOf2020PerDay = sumOfYearPerDay(data, 2020)
sumOf2020PerDay_long_df = pd.DataFrame(sumOf2020PerDay, columns=['location', 'month', 'day', 'countp'])
marchPed = sumOf2020PerDay_long_df[sumOf2020PerDay_long_df.month == 3]

fig = px.bar(marchPed, x='day', y='countp', color="location")
fig.show()

## Wetter (Bevölk, Regen)